In [3]:
import glob, sys, os, re, shutil, h5py, corner
import scipy.io
# import logging
#logging.basicConfig(filename='rahul_prompt_BH_collapse.log', level=logging.DEBUG)
# logging.basicConfig(handlers=[logging.FileHandler('rahul_prompt_BH_collapse.log', 'w', 'utf-8')])
# logging.info('logfile of outputs of rahul_prompt_BH_collapse.ipynb')
import pandas as pd
pd.options.display.max_colwidth = 200
pd.set_option('display.expand_frame_repr', False)
from pathlib import Path
from shutil import copyfile

import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import rc
import matplotlib.pyplot as plt
#plt.style.use('./rahulkashyap.mplstyle')
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from matplotlib import rc
##from IPython.display import

import numpy as np
from scipy.interpolate import splev, splrep, make_interp_spline, BSpline
from scipy.optimize import curve_fit
from scipy import optimize
from scipy import stats
# import statsmodels.api as sm
# from statsmodels.sandbox.regression.predstd import wls_prediction_std
from scipy.stats import truncnorm
from astropy import units

# import units

# def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
# return truncnorm(
# (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [4]:
sys.path.append('/home/nidhi/DDP/Prompt_Collapse_Data/bnspromptcollapse_UM/data/EOSBHB')
import units
# import myconstants as const
# from lib_pbh import *

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy, pylab

colors = ["#3690c8","#f8db37","#ebab78","#e4607b","#f2efed","#8fb5c7","#352c30","#c0d3e0","#eddab1","#d99fad"]

import seaborn as sns
sns.set_context('talk') 

sns.set_theme(font_scale=1.2)
sns.set_palette('colorblind')
sns.set_style('ticks')

pylab.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'axes.grid' : True,
        'grid.linestyle' : ':',
        'grid.color' : '#ececec'

    }
)

pylab.rcParams['axes.linewidth'] = 1
pylab.rcParams['axes.prop_cycle'] = pylab.cycler(color=colors)

In [5]:
# Constants
parsec_m = 3.08567758149e16     # 1 parsec in meters
megaparsec_m = 1e6 * parsec_m   # 1 megaparsec in meters
lightyear_m = 9.4607e15         # 1 light year in meters
c = 299792458

def get_retarded_time(time):
    distance = 40*megaparsec_m
    retrd_time = time - (distance/c)
    return retrd_time

def get_data(path):
    if(glob.glob(path+'waveforms/tmerger.dat')):
        tmrg = np.loadtxt(path+'waveforms/tmerger.dat',dtype='float')
    else:
        tmrg=0
        print("\x1b[31m\" setting tmrg=0 \"\x1b[0m") 

    retarded_time = get_retarded_time(tmrg)
    h22 = np.loadtxt(path+'waveforms/strain_l2_m2.dat',dtype='float',comments='#',usecols=[0,1,2,4])
    lapse=np.loadtxt(path+'collated/alp.minimum.asc',dtype='float',comments='#',usecols=[1,2])
    dens_max = np.loadtxt(path+'collated/rho.maximum.asc',dtype='float',comments='#',usecols=[1,2])
    HNorm2=np.loadtxt(path+'collated/H.norm2.asc',dtype='float',comments='#',usecols=[1,2])

    return tmrg, retarded_time, h22, lapse, dens_max, HNorm2

simulation = 'EOSBHB_M1050-1750_LR_MF'
path = '/home/nidhi/DDP/Prompt_Collapse_Data/bnspromptcollapse_UM/data/EOSBHB/' + simulation + '/'
tmrg, retarded_time, h22, lapse, dens_max, HNorm2 = get_data(path)

# Example: Define unit conversion constants
c = 2.99792458e10       # cm/s (speed of light in CGS)
G = 6.67430e-8          # cm^3 g^-1 s^-2 (gravitational constant)
M_sun = 1.98847e33      # grams (solar mass)

# Time unit in cactus is usually GM/c^3
time_unit = G * M_sun / c**3  # in seconds

def conv_time(from_unit, to_unit, time_array):
    # Assuming from_unit = 'cactus', to_unit = 'cgs'
    # So we convert GM/c^3 -> seconds
    return time_array * time_unit

time_data = conv_time('cactus', 'cgs', h22[:, 0] - tmrg) * 1e3
h22_data = np.real(h22[:, 2]) # The real part of the strain data

tmrg_index = np.where(time_data >= 0)[0][0] # Find the index where time >= 0
sliced_time = time_data[tmrg_index:] # Slice time data
sliced_h22 = h22_data[tmrg_index:] # Slice strain data


In [9]:
def damped_sinusoid1(t, A1, f1, phi1, tau1):
    tau1 = max(tau1, 1e-3)  # avoid division by zero
    t = t - t[0]            # normalize time to start at 0
    return A1 * np.cos(2 * np.pi * f1 * t + phi1) * np.exp(-t / tau1)

def fit_until_match(time_data, waveform_data, model_func, fixed_initial, threshold=0.9999):
    best_match = -np.inf
    best_params = None
    best_fit = None

    bounds = (
        [0, 0.01, -np.pi, 1e-3],     
        [10*np.max(waveform_data), 10000.0, np.pi, 1e3]  
    )

    for i in range(0, len(time_data) - 200):
        sliced_t = time_data[i:i+200]
        sliced_data = waveform_data[i:i+200]
        sliced_t -= sliced_t[0]

        try:
            params, _ = curve_fit(
                model_func,
                sliced_t,
                sliced_data,
                p0=fixed_initial,
                bounds=bounds,
                maxfev=100000
            )
            fit = model_func(sliced_t, *params)
            match = np.corrcoef(sliced_data, fit)[0, 1]

            if match > best_match:
                best_match = match
                best_params = params
                best_fit = fit

            if match >= threshold:
                print(f"Match of {match:.5f} found at index {i}")
                return sliced_t, sliced_data, best_fit, best_params

        except Exception:
            continue

    print("No high match found.")
    return None, None, None, None

# --- Simulated waveform generation (replace with real data) ---
t = np.linspace(0, 1, 10000) * 1e-3  # 1 ms total, in seconds
true_params = [1.0, 3000, 0.0, 0.002]  # A1, f1, phi1, tau1
signal = damped_sinusoid1(t, *true_params)
noise = 0.05 * np.random.randn(len(t))
waveform = signal + noise

# Initial guess
fixed_initial_guess = [0.8, 2800, 0.1, 0.003]

# Run fit
sliced_t, sliced_data, fitted_data, best_params = fit_until_match(
    t, waveform, damped_sinusoid1, fixed_initial_guess
)

# Plot if successful
if fitted_data is not None:
    plt.figure(figsize=(10, 5))
    plt.plot(sliced_t * 1e3, sliced_data, label='Data')
    plt.plot(sliced_t * 1e3, fitted_data, label='Best Fit')
    plt.xlabel('Time (ms)')
    plt.ylabel('Strain')
    plt.title('Best Fit with Match ≥ 99.99%')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("Fit was unsuccessful.")

No high match found.
Fit was unsuccessful.
